<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

<br><h2>Script 1 | Efficient Base Modeling</h2>
<h4>DAT-5390 | Computational Data Analytics with Python</h4>
Chase Kusterer - Faculty of Analytics<br>
Hult International Business School<br><br><br>

<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

<br><br>
In this script, we will:

* Refresh our knowledge of Python and important analytical concepts.
* Learn how to efficiently develop a base model using continuous features from the Ames Housing dataset.

<h2>Part I: Imports and Path</h2><br>

<strong>a) Import the following packages:</strong>
* pandas (as pd)
* seaborn (as sns)
* matplotlib.pyplot (as plt)

Then, import <em>ames_continuous</em> dataset using pd.read_excel().

In [ ]:
# importing libraries
_____ pandas _____ pd                  # data science essentials
import matplotlib.pyplot as plt        # essential graphical output
import _____ as sns                    # enhanced graphical output
import warnings                        # warnings
import statsmodels.formula.api as smf  # classical statistical modeling
from sklearn.model_selection import train_test_split # train/test split
import sklearn.linear_model            # faster linear modeling
from os import listdir                 # NEW! paths and directories
from baserush.optimize import quick_lm # NEW! efficient base modeling

# setting pandas print options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', lambda x: '%.2f' % x)


# suppressing warnings
warnings.filterwarnings(action = 'ignore')


# specifying file name
file = './____/ames_continuous.xlsx'


# reading the file into Python
housing = _____


# outputting the first ten rows of the dataset
housing._____

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<h3>Breaking Down the Path</h3><br>
There is tremendous value in understanding the path structure your computer uses to find things. Such knowledge is transferable across a wide variety of technical applications. Our current path is as follows:
<br><br>

~~~
./datasets/ames_continuous.xlsx
~~~

<br><br>
The syntax of this path can be interpreted as follows:
<br><br>

~~~
[FOLDER WHERE THIS NOTEBOOK IS] / [FOLDER WHERE THE DATASET IS] / [EXCEL FILE NAME].xlsx
~~~

<br><br>
If we were to write the path in human language, it would appear as follows:
<br><br><br>

~~~
Start in the folder where this Notebook is located...

\ and then

Navigate into the folder named "datasets", which is located in the same place as this Notebook... 

\ and then

Select the file named "ames_continuous.xlsx"

~~~

<br><br>
The contents of each part of the directory can be observed with the help of the <a href="https://docs.python.org/3/library/os.html#os.listdir">list directory method</a> coming from <a href="https://docs.python.org/3/library/os.html">the os package</a>.

In [ ]:
# calling help on listdir (from the os package)
help(listdir)

<br>

In [ ]:
# everything in this Notebook's folder (current directory)
for item in listdir(path="."): # one dot
    print(item)

<br>

In [ ]:
# going backwards in the path (parent directory)
for item in listdir(path=".."): # two dots
    print(item)

<br>

In [ ]:
# checking what's in the datasets folder
for item in listdir(path="./datasets"): # ./[folder name]
    print(item)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<h4>b) Navigate to the <em>script_images</em> folder.</h4>

In [ ]:
# printing all files in the script images directory
_____

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

It looks like there's a .gif in this folder. Let's check it out!

<h4>c) Complete the code and copy/paste it into the markdown cell below.</h4><br>

~~~
![dude_gif](./script_images/_____._____)
~~~


~~~
--------------------------------------
 CLICK HERE TO OPEN THE MARKDOWN CELL 
--------------------------------------
~~~

<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

<h3>Part II: Initial Exploration of the Dataset</h3><br>

<h4>a) How many observations (rows) are present in the dataset? How many features (columns)?</h4><br>
Use the following code to complete the formatted string (an f-string) that prints the number of observations and the number of features.

In [ ]:
# formatting and printing the dimensions of the dataset
print(f"""
Size of Original Dataset
------------------------
Observations (rows): {housing.shape[0]}
Features (columns) : {housing.shape[1]}
""")

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<h4>b) Access general information about each feature, including data types and the number of non-missing values.</h4>

In [ ]:
# INFOrmation about each variable
housing.info(verbose = True)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
The dataset has missing values. This is definitely something of interest that we will take care of in a later script. For now, we will drop them to move forward with our initial analysis.

In [ ]:
help(housing.dropna)

<br>

In [ ]:
# observations before dropping missing values
print(f"Observations before drop: {len(housing)}")

<br>

In [ ]:
# dropping missing values
housing = housing.dropna(axis = 0)

# observations after dropping missing values
print(f"Observations after drop: {len(housing)}")

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
<h3>Analyzing the Distribution of Sale Prices</h3><br>
Notice how the Y-variable (<em>&nbsp;Sale_Price&nbsp;</em>) is encoded as an integer as it does not contain decimal places. While this is common in real estate pricing, it introduces a violation of continuity, which is important for predictive models like linear regression. A truly continuous variable should, in theory, have an infinite range and have an infinite number of decimal places. Our Y-variable violates does not meet either condition. However, we must keep in mind that statistics and real-world applications are expected to have a certain degree of misalignment.
<br><br>
This is one of the many reasons that we do not expect our predictions to be perfect (for example, our predicted sale prices will have decimal places). We do, however, expect to develop a general understanding as to what features affect the sale price of a house in Ames, Iowa. The word <em>general</em> is important as base models are often built using one of several <a href="https://en.wikipedia.org/wiki/Generalized_linear_model">generalized linear models</a>.
<br><br>
Note that a <strong>y-variable</strong> is often referred to as a <strong>response variable</strong> or a <strong>dependent variable</strong>. Think of this in terms of the following:<br>

* Question: How much is the sale price of a home in Ames, Iowa?<br>
* <strong>Response</strong>: Well, it <strong>depends</strong> on the features of each house.

<br>
Additional names for the X- and y-variables can be found in <a href="https://www.statsmodels.org/stable/endog_exog.html">the User Guide for statsmodels</a>.
<br><br>
Next, we will use a histogram to visualize <em>Sale_Price</em>. We are hoping to find a normal distribution that is symmetrical. Symmetry is important for modeling with straight lines, as in linear regression.
<br>
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<br>
<h4>e) Develop a histogram to analyze the distribution of the Y-variable.</h4><br>
Does it look as if this variable is normally distributed? Does it appear to be skewed positive or negative? The following help(&nbsp;) file may be useful in completing this task.

In [ ]:
# documentation for making histograms with seaborn
help(sns.histplot)

<br>

In [ ]:
# developing a histogram using HISTPLOT
sns.____(data  = ____,
         x     = ____,
         kde   = True)


# title and axis labels
plt.title(label   = "Distribution of Housing Sale Prices")
plt.xlabel(xlabel = "Sale Price") # avoiding using dataset labels
plt.ylabel(ylabel = "Count")


# displaying the histogram
____.____()

<br>
As can be observed from the histogram above, sale prices are skewed positive. This also something of interest that we will take care of in a later script. For now, let's move forward as the distribution of sale prices appear to be relatively normal. 

<h4>a) Complete the code below to generate descriptive statistics, rounded to two decimal places.</h4>

In [ ]:
# descriptive statistics for numeric data
housing_stats = housing.iloc[ : , 1: ]._____(include = 'number')._____


# checking results
housing_stats

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

Let's subset the results to focus on the distributions of each feature. Do we have enough variance in each feature to use it in predictive modeling?

In [ ]:
# analyzing feature distributions
housing_stats.iloc[ 3: , : ].round(decimals = -2) # negative rounding

<br>

Everything is looking good with the exception of pool areas (&nbsp;<em>Pool_Area</em>&nbsp;). The dataset might not have enough houses with pools for this feature to be useful in base modeling.

<h4>b) Create a frequency table for <em>Pool_Area</em> using value_counts(&nbsp;).</h4>

In [ ]:
# frequency table for Pool_Area
_____

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
<strong>c) Modify the code to show:</strong>
    
* True if a pool area is greater than zero.
* False if a pool area is equal to zero.

In [ ]:
# house has pool True or False


<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

There aren't enough houses with pools in the dataset. Thus, we will need to drop <em>Pool_Area</em> until more data is collected. This is also a good time to drop <em>property_id</em> as it does not serve an analytical purpose in our situation.

In [ ]:
# dropping Order and Pool_Area
housing.drop(columns = ['property_id', 'Pool_Area'],
             axis    = 1,
             inplace = True,
             errors  = 'ignore')

<br>

In [ ]:
# checking results
housing.columns

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
<h2>Part III: Base Modeling</h2><br>
It's time to develop a base model. As a review, base models are very important as they:

* Allow us to confirm our original (base) data, assumptions, and domain knowledge. Think of this as a common sense test. If the findings from our base model don't make sense, we likely need new data, different assumptions about the relationships between our features, or better domain knowledge before moving forward.
* Provide a benchmark to compare to more complex models. Additionally, as models get more complex, they tend to get less interpretable and even harder to take action from.
* Are built with features that follow the assumptions of the type of model we are using. This will be discussed in more detail in class.

<br>
Base modeling will also help us understand the value of the analytical techniques covered throughout this course (missing value analysis, feature engineering, etc.). To get started, let's analyze the linear correlations between our <em>Sale_Price</em> and our X-features. This will help us find good X-candidates for our model.
<br>
<h4>a) Complete the code below and analyze the correlations with <em>Sale_Price</em>.</h4>

In [ ]:
# developing a correlation matrix
housing_corr = housing.____(method = 'pearson')


# filtering results to show correlations with Sale_Price
housing_corr.loc[ : , ____].round(decimals = 2).sort_values(ascending = False)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<h4>b) Develop a scatter plot between <em>Sale_Price</em> and the feature with the strongest correlation to <em>Sale_Price</em>.</h4>

In [ ]:
# setting figure size
fig, ax = plt.subplots(figsize = (9, 6))


# developing a scatterplot
sns.____(x    = ____,
         y    = ____,
         data = ____)


# SHOWing the results
____.____()

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<h3>Building A Base Model</h3><br>
The following code has been provided for you. Its purpose is to provide a basic framework for developing a predictive model in Python using the <a href="https://www.statsmodels.org/stable/index.html">statsmodels</a> package. Keep in mind that there are several techniques we can employ to make this model more optimal, which we will cover in our later sessions.

In [ ]:
## using the statsmodels package ##

# Step 1: INSTANTIATE a model object
lm_best = smf.ols(formula = """Sale_Price ~ Gr_Liv_Area""",
                  data = housing)


# Step 2: FIT the data into the model object
results = lm_best.fit()


# Step 3: analyze the SUMMARY output
print(results.summary())

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
<h2>Team Challenge</h2>

<h4>a) Develop an optimal base model using more than one x-feature.</h4><br>
Your task is to find the combination of X-features that maximizes adjusted R-squared, where all coefficients have p-values of $\leq$ 0.05.
<br><br><br>
<strong><u>Tips</u></strong>

* A common approach is to start with all of the X-features in the model and remove insignificant ones one at a time (also known as backward selection).
* If a feature is removed from a model, expect the p-values for other features to change. Try not to remove too many at a time and test out different combinations.

In [ ]:
# Step 1: INSTANTIATE a model object
lm_best = smf.ols(formula =  """Sale_Price ~ ____ +
                                             ____ +""",
                                data = housing)


# Step 2: FIT the data into the model object
results = lm_best.fit()


# Step 3: analyze the SUMMARY output
print(results.summary())

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
<h2>Improving Our Base Modeling Approach</h2>

Our current base modeling approach using <em>statsmodels</em> is excellent for analyzing the statistical side of models, including reliability indicators like p-values. However, this approach does not take <strong>model stability</strong> into account, which is must faster to analyze using tools from <em>scikit-learn</em>.
<br><br><br>
<strong>b) Complete the following code cells to check for model stability.</strong>

In [ ]:
# setting x-data
x_base = _____


# preparing y-data
original_y = 'Sale_Price'

<br>

In [ ]:
# preparing x-data
x_data = housing[ _____ ]


# preparing y-data
y_data = housing[ _____ ]


# train-test split
x_train, x_test, y_train, y_test = train_test_split(x_data,
                                                    y_data,
                                                    test_size    = 0.25,
                                                    random_state = 702 )

<br>

In [ ]:
# naming the model
model_name = "Linear Regression"


# INSTANTIATING model object
model = sklearn.linear_model.LinearRegression()


# FITTING to training data
model_fit = model.fit(x_train, y_train)


# PREDICTING on new data
model_pred = model.predict(x_test)


# SCORING results (R-Square)
model_train_score = round(model.score(x_train, y_train), ndigits = 4)
model_test_score  = round(model.score(x_test, y_test), ndigits = 4)
model_gap         = round(abs(model_train_score - model_test_score), ndigits = 4)


# displaying results
print('Training Score :', model_train_score)
print('Testing Score  :', model_test_score)
print('Train-Test Gap :', model_gap)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

As can be observed from the train-test gap above, the base model is unstable. As a reminder, a general stability rule is that the train-test gap should be less than or equal to 0.05. Our current approach would require us to build another model in <em>statsmodels</em>, check for significance with p-values, and then check for stability via the train-test gap. However, we can apply a different methodology that was designed specifically for quick and stable base modeling.

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
<h2>Part IV: Efficient Base Modeling with BaseRush</h2><br>

As its name implies, <a href="https://pypi.org/project/baserush/">BaseRush</a> was designed to speed up the base modeling process. Throughout our course, will see this package in action as we endeavor to build better models, keeping stability at the heart of our decision making process. For linear models, BaseRush utilizes a technique called <a href="https://towardsdatascience.com/model-selection-in-linear-regression/">stepwise selection</a>, which can be broken down as follows:

1. <strong>Start</strong> with an empty model (no x-features).
2. <strong>Forward Step:</strong> Add in the x-feature with the smallest p-value.
3. <strong>Forward Step:</strong> Add in the x-feature with the second smallest p-value.
4. <strong>(potential) Backward Step:</strong> Since p-values change with the addition of new features, if any p-values are beyond the p-value threshold (default = 0.05), remove the x-feature with the highest p-value.
5. <strong>Keep alternating forward and backward steps</strong> until there are no more acceptable x-features available.

<br>
Notes:

* All forward steps assume there is an x-feature with a p-value less than or equal to the p-value threshold (default = 0.05).
* Only one x-feature is removed per backward step, but multiple backward steps can occur in sequence.

<h4>a) Run the codes below and analyze their results.</h4>

In [ ]:
# checking documentation
help(quick_lm)

<br>

In [ ]:
# preparing x-data
x_all = housing.drop('Sale_Price', axis = 1)


# preparing y-data
original_y = housing[ 'Sale_Price' ]


# stepwise selection
quick_model = quick_lm(x_data = x_all,
                       y_data = original_y,
                       threshold_in  = 0.01,
                       threshold_out = 0.05,
                       test_size     = 0.25)

<br>

In [ ]:
# x-feature results
quick_model['selected_features']

<br>

In [ ]:
# stepwise selection process
quick_model['history']

<br>

In [ ]:
# model object
quick_model['model']

<br>

In [ ]:
# summary statistics (statsmodels)
quick_model['model'].summary()

<br>

<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

~~~
 _    _      _                          
| |  | |    | |                         
| |  | | ___| | ___ ___  _ __ ___   ___ 
| |/\| |/ _ \ |/ __/ _ \| '_ ` _ \ / _ \
\  /\  /  __/ | (_| (_) | | | | | |  __/
 \/  \/ \___|_|\___\___/|_| |_| |_|\___|
                                        
                                        
______            _    _ _ _            
| ___ \          | |  | | | |           
| |_/ / __ _  ___| | _| | | |           
| ___ \/ _` |/ __| |/ / | | |           
| |_/ / (_| | (__|   <|_|_|_|           
\____/ \__,_|\___|_|\_(_|_|_)           
                                        
~~~

<br><br><hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
<h2>Review: statsmodels OLS Study Sheet</h2><br>
Below is a summary of the OLS regression output from statsmodels.
<br>

<br><br>
<div style = "width:image width px; font-size:80%; text-align:center;">
<br>
<img src="./script_images/statsmodels_OLS_output_2.png" width="800" height="500" style="padding-bottom:0.5em;"><em>Figure 1a: statsmodels OLS Regression Output Study Sheet - Part I</em>
<br><br><br><hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br><br>
<img src="./script_images/statsmodels_OLS_output_3.png" width="800" height="500" style="padding-bottom:0.5em;"><em>Figure 1b: statsmodels OLS Regression Output Study Sheet - Part II</em>
</div>

<br>